In [3]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv('Cleaned_Data.csv', low_memory=False)

In [8]:
df = df.sort_values(by=['distinct_id', 'time'])
df['ae_session_length'] = df['ae_session_length'].fillna('-1')
df['ae_session_length'] = df['ae_session_length'].str.replace(',', '')
df['ae_session_length'] = df['ae_session_length'].astype(float)
df.head()

,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,...,ios_lib_version,ae_first_app_open_date,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown
83729,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-02-13 15:52:50.000000,-1.0,Mumbai,Maharashtra,IN,1,1,NaN,...,NaN,NaN,NaN,NaN,INTL,0,0,0,0,0
62403,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-03-08 20:43:54.000000,-1.0,Ahmedabad,Gujarat,IN,1,1,NaN,...,NaN,NaN,NaN,NaN,INTL,0,0,0,0,0
84492,AppOpen,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:32.000000,-1.0,Palo Alto,California,US,1,0,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
84494,Visit Signup/Login Page,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:35.000000,-1.0,Palo Alto,California,US,1,1,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
84495,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:49:02.000000,30.1,Palo Alto,California,US,1,1,NaN,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0


## Unique Events

In [9]:
events = df['name_x'].unique()
events.sort()
events

array(['$ae_crashed', '$ae_first_open', '$ae_session', '$ae_updated',
       '$campaign_bounced', '$campaign_delivery', '$message_suppressed',
       '$unsubscribe', 'AppOpen', 'Collection - Tap Follow Button',
       'Collection - Tap Restaurant', 'Collection - Tap Return Button',
       'Collection - Tap Search Bar',
       'Collection - Visit Collection Detail Page',
       'Collection Flow - Create New Collection',
       'Collection Flow - Tap Collection',
       'Collection Flow - Tap New Collection', 'DMV-resident',
       'Delivery Page - Visit For 10 Sec', 'Edit Rating - Add Photo',
       'Edit Rating - Delete', 'Edit Rating - Edit', 'Edit Rating - Exit',
       'Edit Rating - Modify Date', 'Explore - Tap Collection',
       'Explore - Visit Explore Page', 'Feed - Bookmark Icon',
       'Feed - Maps Icon', 'Feed - Scroll Down', 'Feed - Tap Dish',
       'Followers Page - Go to Profile', 'Home - Explore Nearby',
       'Home - Order Delivery', 'Invite Page - Successful Text',


## Create a new dataframe with only the columns we need to categorize events

In [13]:
# short_df only includes the event name and an empty column for category

short_df = df[['name_x']]
short_df.loc['category'] = ''

/Users/graceqiu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


## Dictionary of Weights for each Category

In [18]:
weight_dict = {
    'Basic' : 1,
    'Profile' : 2,
    'Explore' : 3,
    'Menu' : 3,
    'Collection' : 3,
    'Rating' : 4,
    'Share' : 5,
    'MISC' : 1,
    'AE' : 1
}

## Dictionary of Categories and their Corresponding Events

In [19]:
event_dict = {
    'Basic' : ['Login', 'Logout', 'AppOpen', 'Onboarding - Allow Location', 'Onboarding - Allow Notifications', 
               'Onboarding - Always Allow Location', 'PageVisit', 'Permissions for Locations', 
               'Permissions for Notifications', 'Signup - Sign In', 'Signup - Skip', 'Signup Success', 
               'Visit Signup/Login Page',  'Visit Home Page'],
    
    'Profile' : ['Password Changed', 'Profile - Change Section', 'Profile - Go to Collection', 
                 'Profile - Go to Menu', 'Profile Updated', 'Sidebar - Change Password', 
                 'Sidebar - Contacts Permissions', 'Sidebar - Edit Profile', 'Sidebar - Go to Followers', 
                 'Sidebar - Privacy Policy', 'Sidebar - Sign Out', 'Sidebar - Terms and Conditions', 
                 'Visit Change Password Page', 'Visit Edit Profile Page', 'Followers Page - Go to Profile', 
                 'Item Page - Go to Profile'],
    
    'Explore' : ['Explore - Tap Collection', 'Explore - Visit Explore Page', 'Home - Explore Nearby', 
                 'Feed - Bookmark Icon', 'Feed - Maps Icon', 'Feed - Scroll Down', 'Feed - Tap Dish', 'Item Page - Bookmark', 
                 'Landing - Go to Delivery Feed', 'Landing - Go to Walking Feed', 'Search - Change Section', 
                 'Search - Follow User', 'Search - Invalid Location', 'Search - Results Loaded', 'Search - Tap Home Bar', 
                 'Search - Tap Nearby Restaurant', 'Search - Tap Result', 'Search - Tap Return', 'Search - Tap to Request', 
                 'Search - Unfollow User', 'Tap Searched Dish', 'Visit Item Page', 
                 'Visit Menu Page'],
    
    'Menu' : ['Maps Popup - Go to Reviews', 'Maps Popup - Maybe Later', 'Maps Popup - Open', 'Menu Page - Accuracy Popup', 
              'Menu Page - Add New Dish', 'Menu Page - Collection via Add Icon', 'Menu Page - Filter Items', 
              'Menu Page - Go to Item Page', 'Menu Page - Rate via Add Icon', 'Menu Page - See Restaurant Details', 
              'Menu Page - Submit Flag', 'Menu Page - Switch Menus', 'Menu Page - Tap Flag Icon', 'Visit Menu Page', 
              'Nearby - Bookmark Dish', 'Nearby - Delivery Link', 'Delivery Page - Visit For 10 Sec', 'Home - Order Delivery', 
              'Nearby - Open Maps', 'Nearby - Tap Dish Maps', 'Visit Carousel Wise Dish List'],
    
    'Rating' : ['Edit Rating - Add Photo', 'Rating Flow - Complete', 'Rating Flow - Rate', 
                'Rating Flow - Resubmit', 'Rating Flow - Start', 'Request Restaurant', 
                'Edit Rating - Delete', 'Edit Rating - Edit', 'Edit Rating - Exit', 
                'Edit Rating - Modify Date', 'Item Page - Open My Rating', 
                'Item Page - Tap Upload Photo', 'Profile - Edit Rating Popup'],
    
    'Collection' : ['Collection - Tap Follow Button', 'Collection - Tap Restaurant', 'Collection - Tap Return Button', 
                    'Collection - Tap Search Bar', 'Collection - Visit Collection Detail Page', 
                    'Collection Flow - Create New Collection', 'Collection Flow - Tap Collection', 
                    'Collection Flow - Tap New Collection'], 
    'Share' : ['Share Page - Instagram Story', 'Share Page - Skip', 'Provided App Feedback', 
               'Sidebar - Go to Feedback', 'Sidebar - Invite a Friend',  'Visit App Feedback Page', 
               'Invite Page - Successful Text', 'Search - Invite Button'],
    
    'AE' : ['$ae_crashed', '$ae_first_open', '$ae_session', '$ae_updated', '$campaign_bounced', 
            '$campaign_delivery', '$message_suppressed', '$unsubscribe'],
    
    'MISC': ['DMV-resident'],
}

## Mapping Event Dictionary to Dataframe

In [25]:
# create a reverse events dictionary in order to use map function 

reverse_dict = {}
for key, value in event_dict.items():
    for event in value:
        reverse_dict.setdefault(event, []).append(key)

In [26]:
# apply reverse dictionary to new column called category that corresponds to the event name

short_df['category'] = short_df['name_x'].map(reverse_dict)

/Users/graceqiu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
# get the first item in category because each value is a list of one element

short_df['category'] = short_df['category'].str[0]

/Users/graceqiu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
# map the weights for each category to a score column

short_df['score'] = short_df['category'].map(weight_dict)

/Users/graceqiu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
short_df.head()

,name_x,category,score
83729,$ae_first_open,AE,1.0
62403,$ae_first_open,AE,1.0
84492,AppOpen,Basic,1.0
84494,Visit Signup/Login Page,Basic,1.0
84495,$ae_session,AE,1.0


## Merge Dataframes

In [31]:
# drop 'name_x' column from short_df and then append to the original dataframe

short_df = short_df.drop(columns = ['name_x'])
df = pd.concat([df, short_df], axis=1)
df.head()

,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,FirstTimeUploaded,...,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown,category,score
0,$ae_session,Gh@g.com_5858,2019-08-19 21:47:54.000000,179.7,Bengaluru,Karnataka,IN,1.0,0.0,NaN,...,NaN,NaN,INTL,0.0,0.0,0.0,0.0,0.0,AE,1.0
1,$ae_session,A5C759A6-9FE5-4071-9D0A-4B91C61F63BD,2020-01-12 07:35:29.000000,16.3,Pasadena,California,US,1.0,0.0,NaN,...,NaN,NaN,US,1.0,0.0,0.0,0.0,0.0,AE,1.0
2,Search - Change Section,4E1A70A0-C2DB-46EB-A9C8-B4F0B0815771,2020-01-12 16:36:27.000000,-1.0,Kyiv,Kyiv City,UA,1.0,0.0,NaN,...,119 days 05:44:04.000000000,203.72842,INTL,0.0,1.0,0.0,0.0,0.0,Explore,3.0
3,Onboarding - Allow Location,C9434897-9F99-4D5F-B9D3-A3E8DAFC0B74,2020-01-12 16:36:44.000000,-1.0,Mill Valley,California,US,1.0,1.0,NaN,...,NaN,NaN,US,1.0,0.0,0.0,0.0,0.0,Basic,1.0
4,$ae_session,4E1A70A0-C2DB-46EB-A9C8-B4F0B0815771,2020-01-12 16:40:32.000000,106.7,Kyiv,Kyiv City,UA,1.0,1.0,NaN,...,119 days 05:44:04.000000000,203.72842,INTL,0.0,1.0,0.0,0.0,0.0,AE,1.0
